# Testing $T_1$ times for various qubits

In [1]:
import circuitq as cq
import numpy as np
import networkx as nx
import scqubits as sc
import matplotlib.pyplot as plt

## Test 0: LC-Circuit
### Circuit and numerical Implementation

In [2]:
graph = nx.MultiGraph()
graph.add_edge(0,1, element = 'C')
graph.add_edge(0,1, element = 'L');
circuit = cq.CircuitQ(graph)
circuit.get_numerical_hamiltonian(200)
circuit.get_eigensystem();

### $T_1$ time contributions

#### Quasiparticle tunneling

In [3]:
circuit.get_T1_quasiparticles()
print(r'Quasiparticles: T_1=', circuit.T1_quasiparticle)

Quasiparticles: T_1= None


## Test 1: Transmon
### Circuit and numerical Implementation

In [4]:
graph = nx.MultiGraph()
graph.add_edge(0,1, element = 'C')
graph.add_edge(0,1, element = 'J');
circuit = cq.CircuitQ(graph)
circuit.get_numerical_hamiltonian(200)
circuit.get_eigensystem();

### $T_1$ time contributions

#### Quasiparticle tunneling


In [5]:
circuit.get_T1_quasiparticles()
print(r'Quasiparticles: T_1=', circuit.T1_quasiparticle)

Quasiparticles: T_1= 3.616926779871887e+27


## Test 2: Fluxonium
### CircuitQ
#### Circuit

In [ ]:
graph = nx.MultiGraph()
graph.add_edge(0,1, element = 'C')
graph.add_edge(0,1, element = 'J')
graph.add_edge(0,1, element = 'L');

#### Symbolic Hamiltonian

In [ ]:
circuit = cq.CircuitQ(graph)
circuit.h

In [ ]:
circuit.h_parameters

#### Diagonalization

In [ ]:
L = 1e-6
h_num = circuit.get_numerical_hamiltonian(401, parameter_values=[False, False, L, 0])
eigv, eigs = circuit.get_eigensystem()

### SCQubit

In [ ]:
EJ = circuit.c_v["E"]
EC = circuit.c_v["E_C"]
EL = (circuit.phi_0**2) / L
fluxonium = sc.Fluxonium(EJ = EJ, EC = EC, EL = EL, flux = 0, cutoff = 401)
esys = fluxonium.eigensys(evals_count=50)

### Compare

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(np.arange(25), eigv[:25], 'rv', label="CircuitQ")
plt.plot(np.arange(25), esys[0][:25], 'g^', label="SC Qubit")
plt.legend()
plt.xlabel("Eigenvalue No.")
plt.ylabel("Energy")
for n in range(25):
    plt.axhline(esys[0][n], lw=0.5)
plt.ticklabel_format(style='scientific', scilimits=(0, 0))
plt.show() 

In [ ]:
plt.style.use('default')
plt.figure(figsize=(8,6))
def potential(phi):
    return -circuit.c_v["E"]*np.cos(phi/circuit.phi_0) + phi**2/(2*L)
plt.plot(circuit.flux_list, potential(circuit.flux_list), lw=0.7)
for n in range(30):
    plt.plot(circuit.flux_list, 
             eigv[n]+ abs(eigs[:,n])**2*(max(circuit.flux_list)**2/(2*circuit.c_v["L"])) 
             ,label="Eigenstate " +str(n))
plt.legend()
plt.ticklabel_format(style='scientific', scilimits=(0, 0))
plt.xlabel(r"$\Phi$")
plt.ylabel("Energy")
plt.show()

## Test 3: Flux Qubit
### CircuitQ
#### Circuit

In [ ]:
graph = nx.MultiGraph()
graph.add_edge(0,1, element = 'C')
graph.add_edge(0,1, element = 'J')
graph.add_edge(1,2, element = 'C')
graph.add_edge(1,2, element = 'J')
graph.add_edge(0,2, element = 'C')
graph.add_edge(0,2, element = 'J');

#### Symbolic Hamiltonian

In [ ]:
circuit = cq.CircuitQ(graph)
circuit.h_parameters

In [ ]:
circuit.h

#### Diagonalization

In [ ]:
dim = 50
EJ = 2.5*circuit.c_v["E"]
alpha = 0.7
C = circuit.c_v["C"]
phi_ext = np.pi*circuit.phi_0 
h_num = circuit.get_numerical_hamiltonian(dim, parameter_values=[C,C,alpha*C,EJ,EJ,alpha*EJ,phi_ext])
eigv, eigs = circuit.get_eigensystem(100)

In [ ]:
subs_dict = dict()
for n, parameter in enumerate(circuit.h_parameters):
    subs_dict[parameter] = circuit.parameter_values[n] 
circuit.h.subs(subs_dict)

In [ ]:
circuit.h_imp

### SCQubit

In [ ]:
EC = circuit.c_v["E_C"]
fluxqubit = sc.FluxQubit(EJ1 = EJ, EJ2 = EJ, EJ3 = alpha*EJ,
                         ECJ1 = EC, ECJ2 = EC, ECJ3 = EC/alpha,
                         ECg1 = 1e25, ECg2 = 1e25, ng1 = 0, ng2 = 0,
                         flux = phi_ext/(circuit.phi_0*2*np.pi), ncut = int(dim/2))
esys = fluxqubit.eigensys(evals_count=30)

### Compare

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(np.arange(30), eigv[:30], 'rv', label="CircuitQ")
plt.plot(np.arange(30), esys[0][:30], 'g^', label="SC Qubit")
plt.legend()
plt.xlabel("Eigenvalue No.")
plt.ylabel("Energy")
for n in range(25):
    plt.axhline(esys[0][n], lw=0.5)
plt.ticklabel_format(style='scientific', scilimits=(0, 0))
plt.show() 

In [ ]:
def potential(phi_1, phi_2, phi_ex):
    return (-EJ*np.cos(phi_1/circuit.phi_0) - EJ*np.cos(phi_2/circuit.phi_0) -
             alpha*EJ*np.cos((phi_2-phi_1+phi_ex)/circuit.phi_0) )
phis = np.arange(-4*np.pi*circuit.phi_0, 4*np.pi*circuit.phi_0, 8*np.pi*circuit.phi_0/dim)
potential_list = []
for phi_1 in phis:
    for phi_2 in phis:
        potential_list.append(potential(phi_1,phi_2,phi_ext))
plt.style.use('default')
plt.contourf(phis, phis, np.array(potential_list).reshape(dim,dim))
plt.colorbar()
plt.show()

In [ ]:
phi_grid = sc.Grid1d(-4*np.pi, 4*np.pi, dim)
a = fluxqubit.plot_potential(phi_grid=phi_grid);

In [ ]:
for n in range(10):
    fig, ax = fluxqubit.plot_wavefunction(esys=esys, which=n, phi_grid=phi_grid, mode='abs_sqr',figsize=(4,4));
    ax.set_title("Eigenstate " + str(n))
    ax.set_xlabel(r"$\phi_1$")
    ax.set_ylabel(r"$\phi_2$")
plt.show()

In [ ]:
circuit.transform_charge_to_flux()
eigs = circuit.estates_in_phi_basis

In [ ]:
plt.figure(figsize=(15,50))
for n in range(10):
    plt.subplot(5,2, n+1)
    plt.contourf(circuit.flux_list, circuit.flux_list, 
                 abs(np.array(eigs[n].reshape(circuit.n_dim,circuit.n_dim)))**2)
    plt.colorbar()
    plt.title("Eigenstate " + str(n) )
    plt.xlabel(r"$\phi_1$")
    plt.ylabel(r"$\phi_2$")
plt.show()